# Big Data Project

## Configuration

In [ ]:
%%configure -f
{"executorMemory":"8G", "numExecutors":2, "executorCores":3, "conf": {"spark.dynamicAllocation.enabled": "false"}}

In [ ]:
//Denys: val bucketname = "unibo-bd2122-dgrushchak"
//Riccardo: val bucketname = "unibo-bd2223-rbacca"

val bucketname = "unibo-bd2223-rbacca"

//Denys: val path_flights_db = "s3a://"+bucketname+"/project/small_file_10000000.txt"
//Riccardo:val path_flights_db = "s3a://"+bucketname+"/bigdata-project/part_1.txt"

val path_flights_db = "s3a://"+bucketname+"/bigdata-project/xaa" 

sc.applicationId

"SPARK UI: Enable forwarding of port 20888 and connect to http://localhost:20888/proxy/" + sc.applicationId + "/"

In [ ]:
object FlightParser {
    
    val noGenresListed = "(no genres listed)"
    val commaRegex = ",(?=(?:[^\"]*\"[^\"]*\")*[^\"]*$)"
    val pipeRegex = "\\|(?=(?:[^\"]*\"[^\"]*\")*[^\"]*$)"
    val quotes = "\""
    val doubleVerticalLine = "\\|\\|"

    /**
    2 - flightdate (String) ** x._1
    3 - starting airport (String) ** x._2
    4 - destination airport (String) ** x._3
    6 - travel duration (PT2H35M) ** in minutes x._4
    8 - isBasicEconomy (String) ** x._8
    9 - isrefundable (boolean)
    10 - isNonStop (String) ** x._7
    12 - basefare (prezzo base del ticket in $ Double) ** x._9
    12 - seats remaining (Int)
    20 - airline name (company name) ** x._6
    23 - airplane type (Vector of [String||String] ** x._5
    24 - segment distancd (distance in milles: Vector of [Int||Int]  can be [None||None])
    **/
    
    def convertTravelDuration(value: String): Long = {
        val input = value.replace("PT", "")
        val minutes = input.replace("M", "").split("H") //[2, 35]
        
        (minutes(0).trim.toLong)*60 + minutes(1).trim.toLong
    }
    
    def convertAirplaneType(value: String): Array[String] = {
        //Ex. Airbus A320||Boeing 737-800
        val result = value.split(doubleVerticalLine)
        if (result.head == "") Array.empty
        else result
    }
    
    def convertAirlineName(value: String): Array[String] = {
        val result = value.split(doubleVerticalLine)
        if (result.head == "") Array.empty
        else result
    }

    def parseFlightInformationLine(line: String): Option[(String, String, String, Long, Array[String], Array[String], Boolean, Boolean, Double)] = {
        try {
            val input = line.split(commaRegex)
            
            val airplanes = convertAirplaneType(input(23))
            val airline = convertAirlineName(input(21))

            if (airplanes.isEmpty || airline.isEmpty) None
            else Some(input(2), input(3), input(4), convertTravelDuration(input(6)), airplanes, airline, input(10).toBoolean, input(8).toBoolean, input(12).toDouble)
        } catch {
            case _: Exception => None
        }
    }
   
}


In [ ]:
val rddFlights = sc.textFile(path_flights_db).flatMap(FlightParser.parseFlightInformationLine)

## Explorative queries

1. How many distinct airports and aircraft models
2. Average travel duration for airline, only in non-stop flights
3. Percentage of basic economy tickets, based on all tickets
4. Percentage of non-stop flights (flights with one leg)
5. Average and price range of tickets
6. Average ticket price for each airline
7. Average and range of travel distance
8. Top 10 airports with more arriving flights


In [ ]:
sc.getPersistentRDDs.foreach(_._2.unpersist())
val rddFlightsCached = rddFlights.cache()

rddFlightsCached.take(10)

def round(v: Double): Double = {
    (v*100).toInt/100.toDouble
}

val numberOfRecords = rddFlightsCached.count

In [ ]:
//How many distinct airports and aircraft models
val distinctAirports = rddFlightsCached.map(x => x._2).distinct.union(rddFlightsCached.map(x => x._3).distinct).distinct.count
val distinctAircraftModels = rddFlightsCached.flatMap(x => x._5).distinct.count

In [ ]:
//Average travel duration for each airline, in non stop flights
val distinctAirlines = rddFlightsCached.
    filter(x => x._7).
    map(x => (x._6(0), x._4)).
    aggregateByKey((0.0,0.0))((a,v)=>(a._1+v,a._2+1), (a1,a2)=>(a1._1+a2._1,a1._2+a2._2)).
    map({case(k,v)=>(k,v._1/v._2)}).
    collect().foreach(println)

In [ ]:
//Percentage of basic economy tickets, based on all tickets
round((rddFlightsCached.filter(_._8).count.toDouble/numberOfRecords).toDouble*100)

In [ ]:
//Percentage of non-stop flights (flights with one leg)
round((rddFlightsCached.filter(_._7).count.toDouble/numberOfRecords).toDouble*100)

In [ ]:
//Average and price range of tickets
val ticketPrices = rddFlightsCached.map(x => x._9)
println("Min price: " + ticketPrices.min)
println("Max price: " + ticketPrices.max)
print("Avg price: " + round((ticketPrices.sum/numberOfRecords).toDouble))

In [ ]:
// Average ticket price for each airline
val avgTicketPricePerAirline = rddFlightsCached.
    filter(_._7).
    map(x => (x._6(0),(x._9, 1))).
    reduceByKey((a,b) => (a._1+b._1, a._2+b._2)).
    map(m => (m._1,m._2._1/m._2._2)).
    collect.foreach({case (name, value) => println(name + " => " + round(value))})

In [ ]:
// Average and range of travel distance

In [73]:
// Top 10 airports with more arriving flights
val topAirports = rddFlightsCached.
    map(x => (x._3, 1)).
    reduceByKey(_+_).
    sortBy(_._2, false).
    take(10).
    foreach({case (name, value) => println(name + " => " + value)})

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

LGA => 904
LAX => 857
BOS => 704
DFW => 671
DEN => 666
DTW => 638
EWR => 620
ATL => 602
ORD => 559
SFO => 553
topAirports: Unit = ()


## Personal works

> Basta una query compicata a testa

Query approfondite:

- Denys:
    - Aggrego “airline” e “aircraft model” per determinare il prezzo medio di un miglio nautico, e dopo aggrego (MIN) su “aircraft model” per trovare l'airline che vende i biglietti più economici per quel modello.
    - Aggrego su “aircraft model” per calcolare la “travel distance” totale percorsa da ogni modello, poi faccio self-join e aggregazione per determinare il “travel duration” per ogni “aircraft model”. Alla fine determino la velocità di ogni modello partendo dai dati aggregati.
- Riccardo:
    - Aggrego su “aircraft models” per calcolare la classifica discreta dei modelli più usati rispetto a ciascuna “airline”. Eseguo il join col dataset originale, infine riaggrego su “travel duration” e sulla classificazione di prima. Ottengo la durata di media di ogni volo per ogni compagnia (oltre a numero di voli e totale di ore) sul modello di aereo più utilizzato
    - Aggrego su giorno ed airline per calcolare la distanza totale, poi aggrego su airline per calcolare la media dei totali giornalieri (di distanza) per ciascuna compagnia aerea
    - Aggrego su aeroporto di arrivo e di partenza, calcolando per ogni copia un AVG dei prezzi dei biglietti. Join con dataset originale. Aggrego nuovamente sulla classificazione di prima e sulla distanza totale percorsa. Ottengo così per ogni tratta il prezzo medio e la distanza media percorsa.

In [ ]:
val path_output = "s3a://"+bucketname+"/spark/avgRatPerMovie" //todo